# OpenAgua to WaMDaM to WASH
#### By Adel M. Abdallah, Utah State University, December 2018

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button <img style='display:inline;padding-bottom:15px' src='play-button.png'> on the toolbar above.


# Serve new imported input data from OA into GDX through WaMDaM

# 1. Import python libraries


In [1]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell

import sqlite3
import numpy as np
import pandas as pd
import getpass
from hs_restclient import HydroShare, HydroShareAuthBasic
import os

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot
import sys
from openpyxl import load_workbook

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
import calendar

import sqlite3
import pandas as pd
# https://github.com/NREL/gdx-pandas
from IPython.display import display, Image, SVG, Math, YouTubeVideo
from openpyxl import load_workbook
import gdxpds

# ! pip install gdxpds
# import gdxpds
import gams
from gams import *
from ctypes import c_bool
print 'The needed Python libraries have been imported'

No handlers could be found for logger "gdxpds"


The needed Python libraries have been imported


# 2. Connect to the WaMDaM SQLite on HydroShare
### Provide the HydroShare ID for your resource
Example  
https://www.hydroshare.org/resource/8760f63a1f5545c380a3dd7ec847cb61/
 
resource_id='85e9fe85b08244198995558fe7d0e294'

In [4]:
username = 'amabdallah'
password = 'MyHydroShareWorld'

auth = HydroShareAuthBasic(username=username, password=password)

hs = HydroShare(auth=auth)

print 'Connected to HydroShare'

# Then we can run queries against it within this notebook :)  
resource_url='https://www.hydroshare.org/resource/123b2781b73742a18bfadd21124c6194/' 


resource_id= resource_url.split("https://www.hydroshare.org/resource/",1)[1] 
resource_id=resource_id.replace('/','')

print resource_id

resource_md = hs.getSystemMetadata(resource_id)
# print resource_md
print 'Resource title'
print(resource_md['resource_title'])
print '----------------------------'

resources=hs.resource(resource_id).files.all()

file = ""

for f in hs.resource(resource_id).files.all():

    file += f.decode('utf8')

import json

file_json = json.loads(file)

for f in file_json["results"]:

    FileURL= f["url"]
    
    SQLiteFileName=FileURL.split("contents/",1)[1] 

cwd = os.getcwd()


conn = sqlite3.connect(SQLiteFileName,timeout=10)

print 'Connected to the SQLite file= '+ SQLiteFileName
print 'done'

Connected to HydroShare
123b2781b73742a18bfadd21124c6194
Resource title
Final datasets
----------------------------
Connected to the SQLite file= WEAP_No_OA.sqlite
done


# Connect to SQLite file locallay

In [ ]:

# the SQLite file is published here 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/3SQLite_database/BearRiverDatasets_June_2018.sqlite

WaMDaM_SQLite_Name='WASH_OneYear.sqlite'
# WaMDaM_SQLite_Name='replicateWaMDaM.sqlite'

# Test if the connection works 
conn = sqlite3.connect(WaMDaM_SQLite_Name)
df = pd.read_sql_query("SELECT ResourceTypeAcronym   FROM ResourceTypes Limit 1 ", conn)
print df
                       
print '--------------------'                        
print '\n Connected to the WaMDaM SQLite file called'+': '+ WaMDaM_SQLite_Name

# Query the seasonal data from WaMdaM into a dataframe to write to GDX 

In [ ]:
# The query has hard coded input parameters


# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase_URL="""https://raw.githubusercontent.com/WamdamProject/WaMDaM_JupyterNotebooks/master/3_VisualizeShare/WASH/Query_demand_seasonal.sql
"""

# Read the query text inside the URL
Query_UseCase_text = urllib.urlopen(Query_UseCase_URL).read()


# return query result in a pandas data frame
df_Seasonal_WaMDaM= pd.read_sql_query(Query_UseCase_text, conn)


display (df_Seasonal_WaMDaM)

# Preapre it to Excel

In [ ]:
Instance=df_Seasonal_WaMDaM['InstanceName'][1]
print Instance

In [90]:
df=df_Seasonal_WaMDaM.loc[:, 'SeasonName':'SeasonNumericValue'].T

df=df.loc['SeasonNumericValue':, ]


In [91]:
WASH_ExcelFile='WASH_1yr_InputData.xlsx'
sheetname='demandReq'
#
book = load_workbook(WASH_ExcelFile)

UpdateDemand = book[sheetname]
i =0
for index, row in df.iterrows():
    for j, column in row.iteritems():
        UpdateDemand.cell(row=i+2, column=j+2, value=float(column))
    i += 1
book.save(WASH_ExcelFile)

print 'done writing the value to WASH excel file'

done writing the value to WASH excel file


# Read the WASH Area objective function value

In [92]:
gdx_file = 'C:\Users\Adel\Documents\GitHub\WEAP_WASH_OA\WASH-solution.gdx'
with gdxpds.gdx.GdxFile(lazy_load=False) as f:
    f.read(gdx_file)
    for symbol in f:
        symbol_name = symbol.name
        if symbol_name=='Z':  
            df = symbol.dataframe
            Zvalue=str(df.iloc[0]['Level'])
            print "Z="+ Zvalue


Z=122105.014036


In [ ]:
Z=121526.58605


In [ ]:
Z=121384.065505

Z=122105.014036


In [ ]:
import os
# os.path.dirname(os.path.abspath(__file__))

command="""start cmd cd C:\GAMS\win64/24.7 & gams.exe WASH-CEE6410"""

var=os.system(command)
print var


# Execute GAMS (Before and after update)
https://www.gams.com/latest/docs/API_PY_TUTORIAL.html

In [ ]:
from gams import *

In [ ]:
! conda install -c goop gams 

In [108]:
version = GamsWorkspace.api_version
print version
ws = GamsWorkspace('Test')
# ws = GamsWorkspace(debug=DebugLevel.KeepFiles)
print ws
job = ws.add_job_from_file("C:\Users\Adel\Documents\GitHub\WEAP_WASH_OA\WASH-CEE6410.gms")
job.run()
GamsDatabase = job.out_db

24.5.6
--- Warning: The GAMS version (24.2.3) differs from the API version (24.5.6).


GamsExceptionExecution: GAMS return code not 0 (2), check C:\Users\Adel\Documents\GitHub\WEAP_WASH_OA\Test\_gams_py_gjo0.lst for more details

In [110]:
ws = GamsWorkspace()
job = ws.add_job_from_file("C:\Users\Adel\Documents\GitHub\WEAP_WASH_OA\WASH-CEE6410.gms")
job.run()
GamsDatabase = job.out_db

--- Warning: The GAMS version (24.2.3) differs from the API version (24.5.6).


GamsExceptionExecution: GAMS return code not 0 (2), set the debug flag of the GamsWorkspace constructor to DebugLevel.KeepFiles or higher or define a working_directory to receive a listing file with more details

In [111]:
if len(sys.argv) > 1:
    ws = GamsWorkspace(system_directory = sys.argv[1])
else:
    ws = GamsWorkspace()

GamsException: Invalid GAMS system directory: C:\Users\Adel\Documents\GitHub\WEAP_WASH_OA\-f

In [102]:
GAMSWorkspace.GAMSWorkspace(workingDirectory = null,
systemDirectory = null,
DebugLevel  = DebugLevel.Off 
)

NameError: name 'GAMSWorkspace' is not defined

Z=121526.58605
